In [73]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [27]:
train = pd.read_csv("iris_train.csv")
test = pd.read_csv("iris_test.csv")

In [28]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [29]:
test.head()

,id,species,sepal length (cm),petal length (cm)
0,0,setosa,5.4,1.7
1,1,setosa,5.7,1.5
2,2,setosa,5.3,1.5
3,3,setosa,5.1,1.9
4,4,virginica,6.0,4.8


In [30]:
train['sepal width (cm)'].skew()

0.1896874250525636

In [31]:
train['petal width (cm)'].skew()

-0.593425279337102

In [32]:
from ngboost import NGBRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import *

from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [33]:
lo = LeaveOneOut()
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [34]:
train.groupby('species')['sepal width (cm)'].mean()

species
setosa        3.346667
versicolor    2.790000
virginica     2.933333
Name: sepal width (cm), dtype: float64

In [35]:
train['species'] = train['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2})
test['species'] = test['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2})

In [43]:
X = train.iloc[:, 1:4]
y1 = train.iloc[:, 4]
y2 = train.iloc[:, 5]

In [76]:
target = test.iloc[:, 1:]

In [87]:
ngb_pred1 = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y1))) :
    print(f'{i + 1} FOLD Start!')
    tr_x, tr_y = X.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 100, learning_rate = 0.008, verbose = 10)
    
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 10)
    fold_pred = ngb.predict(target) / 5
    
    ngb_pred1 += fold_pred
    
    print('\n')

1 FOLD Start!
[iter 0] loss=0.4817 val_loss=0.4054 scale=1.0000 norm=0.6661


2 FOLD Start!
[iter 0] loss=0.5259 val_loss=0.2472 scale=1.0000 norm=0.6804


3 FOLD Start!
[iter 0] loss=0.4412 val_loss=0.5676 scale=1.0000 norm=0.6619
== Early stopping achieved.
== Best iteration / VAL61 (val_loss=0.4121)


4 FOLD Start!
[iter 0] loss=0.5237 val_loss=0.2528 scale=1.0000 norm=0.6797


5 FOLD Start!
[iter 0] loss=0.3278 val_loss=1.1351 scale=1.0000 norm=0.6188
== Early stopping achieved.
== Best iteration / VAL81 (val_loss=0.6889)




In [88]:
ngb_pred2 = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y2))) :
    print(f'{i + 1} FOLD Start!')
    tr_x, tr_y = X.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 100, learning_rate = 0.008, verbose = 10)
    
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 10)
    fold_pred = ngb.predict(target) / 5
    
    ngb_pred2 += fold_pred
    
    print('\n')

1 FOLD Start!
[iter 0] loss=1.0230 val_loss=1.1399 scale=2.0000 norm=1.5799


2 FOLD Start!
[iter 0] loss=1.0985 val_loss=0.8686 scale=2.0000 norm=1.6117


3 FOLD Start!
[iter 0] loss=1.0350 val_loss=1.0948 scale=2.0000 norm=1.5798


4 FOLD Start!
[iter 0] loss=1.0504 val_loss=1.0404 scale=2.0000 norm=1.5842


5 FOLD Start!
[iter 0] loss=1.0419 val_loss=1.0637 scale=2.0000 norm=1.5830




In [80]:
lgbm_pred1 = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y1))) :
    print(f'{i + 1} FOLD Start!')
    tr_x, tr_y = X.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    lgbm = LGBMRegressor(random_state = 42, max_depth = 5, n_estimators = 100, learning_rate = 0.01, objective = 'regression', metric = 'mae')
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], eval_metric = 'mae', early_stopping_rounds = 10, verbose = 10)
    fold_pred = lgbm.predict(target) / 5
    
    lgbm_pred1 += fold_pred
    print('\n')

1 FOLD Start!
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 0.287156	valid_1's l1: 0.267841
Early stopping, best iteration is:
[1]	training's l1: 0.292389	valid_1's l1: 0.267386


2 FOLD Start!
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 0.300224	valid_1's l1: 0.21485
[20]	training's l1: 0.295805	valid_1's l1: 0.211799
[30]	training's l1: 0.292941	valid_1's l1: 0.210174
[40]	training's l1: 0.291566	valid_1's l1: 0.20992
[50]	training's l1: 0.29135	valid_1's l1: 0.210716
Early stopping, best iteration is:
[44]	training's l1: 0.291054	valid_1's l1: 0.209825


3 FOLD Start!
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 0.277633	valid_1's l1: 0.317395
[20]	training's l1: 0.275443	valid_1's l1: 0.312527
[30]	training's l1: 0.273461	valid_1's l1: 0.308124
[40]	training's l1: 0.27167	valid_1's l1: 0.304142
[50]	training's l1: 0.270049	valid_1's l1: 0.300541
[60]	training's l1: 0.26928	va

In [79]:
lgbm_pred2 = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y2))) :
    print(f'{i + 1} FOLD Start!')
    tr_x, tr_y = X.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    lgbm = LGBMRegressor(random_state = 42, max_depth = 5, n_estimators = 100, learning_rate = 0.01, objective = 'regression', metric = 'mae')
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], eval_metric = 'mae', early_stopping_rounds = 10, verbose = 10)
    fold_pred = lgbm.predict(target) / 5
    
    lgbm_pred2 += fold_pred
    print('\n')

1 FOLD Start!
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 0.487082	valid_1's l1: 0.602591
[20]	training's l1: 0.45792	valid_1's l1: 0.568558
[30]	training's l1: 0.434797	valid_1's l1: 0.537734
[40]	training's l1: 0.416031	valid_1's l1: 0.509505
[50]	training's l1: 0.398811	valid_1's l1: 0.485006
[60]	training's l1: 0.383227	valid_1's l1: 0.466432
[70]	training's l1: 0.370711	valid_1's l1: 0.448064
[80]	training's l1: 0.359214	valid_1's l1: 0.432147
[90]	training's l1: 0.34862	valid_1's l1: 0.416255
[100]	training's l1: 0.340836	valid_1's l1: 0.404064
Did not meet early stopping. Best iteration is:
[100]	training's l1: 0.340836	valid_1's l1: 0.404064


2 FOLD Start!
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 0.544443	valid_1's l1: 0.405597
[20]	training's l1: 0.506728	valid_1's l1: 0.390531
[30]	training's l1: 0.474066	valid_1's l1: 0.378352
[40]	training's l1: 0.44532	valid_1's l1: 0.367338
[50]	training's l1: 0

In [81]:
submission = pd.read_csv("sample_submission.csv")

In [91]:
submission.iloc[:, 1] = ngb_pred1
submission.iloc[:, 2] = ngb_pred2

In [92]:
submission.to_csv('1019_ngb_1st.csv', index = False)